<a href="https://colab.research.google.com/github/moey920/NLP/blob/master/receipt_transformer_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mxnet-cu101 #
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

In [0]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6ljd8rnu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-6ljd8rnu
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12871 sha256=48e12f69ac64f6228de87017a0505268fdd3fced6801758d2c389b706159c3a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-4_op5f_w/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [0]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [0]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [0]:
##GPU 사용 시
device = torch.device("cuda:0")

In [0]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [0]:
'''
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1
'''

'\n!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1\n!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1\n'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataset_train = nlp.data.TSVDataset("/content/drive/My Drive/캐시카우_노하람인턴_공유폴더/KoBERT/dataset/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/My Drive/캐시카우_노하람인턴_공유폴더/KoBERT/dataset/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [0]:
print(dataset_train[0])

['CJ올리브영( ㈜ 이선신보집', 'CJ올리브영㈜ 인천신포점']


In [0]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [0]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [transform([i[label_idx]]) for i in dataset]
        #self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
        
    def __len__(self):
        return (len(self.labels))


In [0]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [0]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [0]:
import torch
import numpy as np
import torch.utils.data as data

def my_collate(batch):
    #data = torch.stack([item[0].unsqueeze(0) for item in batch], 0)
    data = torch.Tensor([item[0] for item in batch], 0)
    target = torch.Tensor([item[1] for item in batch])
    return [data, target]

class dataset(data.Dataset):
    def __init__(self):
        super(dataset, self).__init__()

    def __len__(self):
        return 100
    
    def __getitem__(self, index):
        return torch.rand(5, 6), list(range(index))

dataloader = data.DataLoader(dataset=data_train,
                      batch_size=batch_size,
                      shuffle=True,
                      collate_fn=my_collate, # use custom collate function here
                      pin_memory=True)

for instance in dataloader:
    print(instance[0].shape, len(instance[1]))
    for labels in instance[1]:
        print('length', len(labels))
    raw_input()

In [0]:
'''
import torch
import numpy as np
import torch.utils.data as data

def my_collate(batch):
    data = torch.stack([item[0].unsqueeze(0) for item in batch], 0)
    target = torch.Tensor([item[1] for item in batch])
    return [data, target]

class dataset(data.Dataset):
    def __init__(self):
        super(dataset, self).__init__()

    def __len__(self):
        return 100
    
    def __getitem__(self, index):
        return torch.rand(5, 6), list(range(index))

dataloader = data.DataLoader(dataset=data_train,
                      batch_size=batch_size,
                      shuffle=True,
                      collate_fn=my_collate, # use custom collate function here
                      pin_memory=True)

for instance in dataloader:
    print(instance[0].shape, len(instance[1]))
    for labels in instance[1]:
        print('length', len(labels))
    raw_input()
    '''

In [0]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [0]:
print(type(test_dataloader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [0]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [0]:
model = BERTClassifier(bertmodel,   dr_rate=0.5).to(device)

In [0]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [0]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [0]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [0]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [0]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [0]:
print(label)

[tensor([[   2,  643, 6973,  ...,    1,    1,    1],
        [   2, 3790, 5483,  ...,    1,    1,    1],
        [   2, 1316, 7993,  ...,    1,    1,    1],
        ...,
        [   2,  702, 4998,  ...,    1,    1,    1],
        [   2,  524,  524,  ...,    1,    1,    1],
        [   2,  717, 3025,  ...,    1,    1,    1]], dtype=torch.int32), tensor([14, 10,  9,  8, 26, 18, 20, 16, 16,  3, 19, 19, 12,  5, 16, 16, 11, 10,
        33, 11, 15, 10, 18, 18,  5,  6,  5,  3,  4, 10, 22, 12, 12, 13, 16, 11,
        11, 14, 25,  9, 18, 16, 14, 24,  6,  8,  8, 12, 13, 19,  7, 13, 49,  6,
        13,  4,  6,  5,  5,  4,  6,  9, 49,  4], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)]


In [0]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [0]:
type(segment_ids)

In [0]:
type(label)

In [0]:
len(label)

3

In [0]:
print(label)

[tensor([[   2,  643, 6973,  ...,    1,    1,    1],
        [   2, 3790, 5483,  ...,    1,    1,    1],
        [   2, 1316, 7993,  ...,    1,    1,    1],
        ...,
        [   2,  702, 4998,  ...,    1,    1,    1],
        [   2,  524,  524,  ...,    1,    1,    1],
        [   2,  717, 3025,  ...,    1,    1,    1]], dtype=torch.int32), tensor([14, 10,  9,  8, 26, 18, 20, 16, 16,  3, 19, 19, 12,  5, 16, 16, 11, 10,
        33, 11, 15, 10, 18, 18,  5,  6,  5,  3,  4, 10, 22, 12, 12, 13, 16, 11,
        11, 14, 25,  9, 18, 16, 14, 24,  6,  8,  8, 12, 13, 19,  7, 13, 49,  6,
        13,  4,  6,  5,  5,  4,  6,  9, 49,  4], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)]


In [0]:
torch.stack(label)

RuntimeError: ignored